# Application
*par Merwan Petelet*

In [ ]:
##### Importation des librairies

In [1]:
pip install moviepy
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
from IPython import display as dis
import pandas as pd
from requests_html import HTMLSession
from moviepy.editor import AudioFileClip
import re
from bs4 import BeautifulSoup
from pylab import specgram, show

In [ ]:
##### Dictionnaire réalisé dans la partie précédente

In [2]:
cheminDuFichierCSV = "./../data/dataframe/BuchananRich.csv"
dico = pd.read_csv(cheminDuFichierCSV, delimiter=",", encoding="utf8")
dico = dico.fillna("")

In [ ]:
##### filtres en widget sur le mot vedette et l'IPA et la taille maximale ainsi que le bouton et les sorties

In [3]:
motVed = widgets.Text(
       value="",
       description='Mot vedette',
       layout={'width': '300px'} )

pronIPA = widgets.Text(
       value="",
       description='pron. IPA',
       layout={'width': '300px'} )

tailleMax = widgets.IntSlider(
min=1,
max=10,
step=1,
description='nbMots',
value = 3
)

# définition du widget button permettant de lancer l'extraction
button = widgets.Button(description='Démarrer')

# définition du widget permettant l'affichage d'une sortie
out1 = widgets.Output()
out2 = widgets.Output()

In [ ]:
##### fonction de scrapping de la 1ère partie avec seulement le lien mp3 de l'audio

In [4]:
def lien(hwd):
    session = HTMLSession()
    urlTarget = "https://dictionary.cambridge.org/dictionary/english/" + hwd
    response = session.get(urlTarget)
    soup = BeautifulSoup(response.text,'html.parser')
    souplettesAudio = soup.findAll('amp-audio')
    if len(souplettesAudio)>0:
        baseURL = "https://dictionary.cambridge.org/"
        audioUK = baseURL + souplettesAudio[0].find('source')["src"]
        audioUS = baseURL + souplettesAudio[1].find('source')["src"]
        return audioUS, audioUK
    else:
        return "Audio introuvable", "Audio introuvable"

In [ ]:
##### fonction utilisée lors du clic sur le bouton

In [5]:
# spécifier les actions à réaliser lors de l'appui sur le bouton
def on_button_clicked(_):
    session = HTMLSession()

    # récupérer le texte saisi par l'utilisateur
    resultat = dico[dico.motVedette.str.contains(motVed.value, na=False,flags=re.IGNORECASE, regex=True)]
    resultat = resultat[resultat.ipa.str.contains(pronIPA.value, na=False,flags=re.IGNORECASE, regex=True)]

    #filtre si plus de mots sont trouvés que le max
    if len(resultat)>tailleMax.value:
        resultat = resultat.sample(tailleMax.value)

    #tableau
    mots = []
    with out1:
        clear_output()
        page = "<table><tr><th width='10%'> mot vedette </th><th width='5%'> nbSyll </th><th width='15%'> pronOrigine </th><th width='10%'> pronSampa </th><th width='10%'> pronIPA </th></tr>"
        for i in range(0, len(resultat)) :
            mots.append(resultat["motVedette"].iloc[i])
            page += "<tr><td align='left'>"+ resultat["motVedette"].iloc[i] +"</td><td>"+ str(resultat["nbSyll"].iloc[i]) +"</td><td>"+ resultat["pron"].iloc[i] +"</td><td>"+ resultat["pronSampa"].iloc[i] +"</td><td>"+ resultat["ipa"].iloc[i] +"</td></tr>"
        page += "</table>"
        display(HTML(page))

    #audios
    with out2:
        clear_output()
        for mot in mots:
            urlUS, urlUK = lien(mot)
            display(HTML("<h3> Mot vedette : " + mot + "</h3><br/>"))
            if(urlUS=="Audio introuvable"):
                display(HTML(urlUS))
            else:
                audio = session.get(urlUS)
                localPathUS = "./../data/audio/" + mot + "US.mp3"
                open(localPathUS, 'wb').write(audio.content)
                audio = session.get(urlUK)
                localPathUK = "./../data/audio/" + mot + "UK.mp3"
                open(localPathUK, 'wb').write(audio.content)
                display(HTML("prononciation US : "))
                dis.display(dis.Audio(url=urlUS))
                audio = AudioFileClip(localPathUS)
                sound_array = audio.to_soundarray()
                specgram(sound_array[:,0])
                show()
                display(HTML("prononciation UK : "))
                dis.display(dis.Audio(url=urlUK))
                audio = AudioFileClip(localPathUK)
                sound_array = audio.to_soundarray()
                specgram(sound_array[:,0])
                show()
                display(HTML("<br/>"))

        

button.on_click(on_button_clicked)

In [ ]:
##### layout des widgets

In [6]:
# les widgets de filtre input
vbFiltres = widgets.HBox([motVed, pronIPA, tailleMax])

# les widgets d'output
vbOuts = widgets.VBox([out1, out2])
hbAllWidgets = widgets.VBox([vbFiltres, button, vbOuts])

In [ ]:
##### affichage de l'application

In [7]:
hbAllWidgets